In [29]:
#import data
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

books = pd.read_csv("/opt/ml/input/code/data/books.csv")
users = pd.read_csv("/opt/ml/input/code/data/users.csv") # books.drop(['img_url', 'img_path'], axis = 1, inplace = True)

isbns = pd.read_csv("/opt/ml/input/code/data/isbn_country.csv")

## Users 결측치

### Users location 세분화

In [23]:
# users 결측치
### location 세분화 -> city, state, county 로
users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거

users['location_city'] = users['location'].apply(lambda x: x.split(',')[0].strip())
users['location_state'] = users['location'].apply(lambda x: x.split(',')[1].strip())
users['location_country'] = users['location'].apply(lambda x: x.split(',')[2].strip())

users = users.replace('na', np.nan) #특수문자 제거로 n/a가 na로 바뀌게 되었습니다. 따라서 이를 컴퓨터가 인식할 수 있는 결측값으로 변환합니다.
users = users.replace('', np.nan) # 일부 경우 , , ,으로 입력된 경우가 있었으므로 이런 경우에도 결측값으로 변환합니다.
###

/tmp/ipykernel_49212/2406416191.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  users['location'] = users['location'].str.replace(r'[^0-9a-zA-Z:,]', '') # 특수문자 제거


### Users location city만 있는 경우 처리

In [24]:
# users 결측치
### city값이 존재하는데, country 정보가 없는 경우 처리
modify_location = users[(users['location_country'].isna())&(users['location_city'].notnull())]['location_city'].values
location = users[(users['location'].str.contains('seattle'))&(users['location_country'].notnull())]['location'].value_counts().index[0]

location_list = []
for location in modify_location:
    try:
        right_location = users[(users['location'].str.contains(location))&(users['location_country'].notnull())]['location'].value_counts().index[0]
        location_list.append(right_location)
    except:
        pass
    
for location in location_list:
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_state'] = location.split(',')[1]
    users.loc[users[users['location_city']==location.split(',')[0]].index,'location_country'] = location.split(',')[2]
    
###

KeyboardInterrupt: 

In [ ]:
users.isna().sum()

user_id                 0
location                0
age                 27833
location_city         122
location_state       1132
location_country      271
dtype: int64

# Books 결측치

In [3]:
# isbn이 잘못된 경우는 없다!!!
books[books['isbn'] != books['img_url'].apply(lambda x: x.split('P/')[1][:10])]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path


In [4]:
books[books['isbn'] != books['img_path'].apply(lambda x: x.split('.')[0][7:])]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path


### Category 대분류 생성

In [5]:
# category 대분류

import re
books.loc[books[books['category'].notnull()].index, 'category'] = books[books['category'].notnull()]['category'].apply(lambda x: re.sub('[\W_]+',' ',x).strip())
books['category'] = books['category'].str.lower()

books['category_low'] = books['category'].copy()

category_high_df = pd.DataFrame(books['category'].value_counts()).reset_index()
category_high_df.columns = ['category', 'count']

other_list = category_high_df[category_high_df['count'] < 5]['category']
books.loc[books[books['category'].isin(other_list)].index, 'category'] = 'others'

books.loc[books[books['category']=='biography'].index, 'category'] = 'biography autobiography'
books.loc[books[books['category']=='autobiography'].index,'category'] = 'biography autobiography'

categories = ['history', 'fiction', 'garden','crafts','physics','adventure','music','fiction','nonfiction','science','science fiction','social','homicide',
 'sociology','disease','religion','christian','philosophy','psycholog','mathemat','agricult','environmental',
 'business','poetry','drama','literary','travel','motion picture','children','cook','literature','electronic',
 'humor','animal','bird','photograph','computer','house','ecology','family','architect','camp','criminal','language','india']

for category in categories:
    books.loc[books[books['category'].str.contains(category,na=False)].index,'category'] = category

In [6]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               67227
category               68851
summary                67227
img_path                   0
category_low           68851
dtype: int64

In [7]:
books['category'].value_counts()

fiction                    41125
others                      5196
biography autobiography     3333
science                     2286
history                     1939
                           ...  
questions and answers          5
internet                       5
grandmothers                   5
caribbean area                 5
ducks                          5
Name: category, Length: 407, dtype: int64

### 데이터 중 특수문자(\&#39;, 문자('\n')을 공백(' ') 전처리)

In [8]:
# summary에 특수문자가 &#39;로 표시된 경우 처리
# 개행 문자('\n')을 공백(' ')으로 변경
summary_ = pd.DataFrame(books[['book_title','summary']])

dic = {
    "&quot;" : '"',
    "&amp;" : "&",
    "&lt;" : "<",
    "&gt;" : '>',
    "&#34;" : '"',
    '&#39;' : "'"
}

for k, v in dic.items():
    summary_['summary'] = summary_['summary'].str.replace(k,v, regex=True)
summary_['summary'] = summary_['summary'].str.replace('\n', ' ')
books['summary'] = summary_['summary']

### book_author 공백 없이 보게 전처리

In [9]:
# book_author 모두 소문자 -> 공백 제거 후 모두 이어붙임
book_author = books['book_author'].str.lower()
books['book_author'] = book_author.map(lambda x: "".join(x.split()))
# books['book_author'].value_counts()

### publisher isbn 기준으로 중복처리

In [10]:
# mission 1
# 같은 출판산데 이름이 다른 경우 처리 a_book 이랑 a_audio
publisher_dict=(books['publisher'].value_counts()).to_dict()
publisher_count_df= pd.DataFrame(list(publisher_dict.items()),columns = ['publisher','count'])

publisher_count_df = publisher_count_df.sort_values(by=['count'], ascending = False)
books['publisher'].nunique() # 수정전 항목 수를 확인합니다.
books[books['isbn'].apply(lambda x: x[:4])=='0140']['publisher'].unique()
modify_list = publisher_count_df[publisher_count_df['count']>1].publisher.values
books['publisher'].nunique() #수정 후 출판사 갯수입니다

11571

## 작가가 썼던 책을 기준으로 language 채우기

In [11]:
# language 결측치 채우기
# 작가가 쓴 책을 찾아서 language 채우기


look_up_df = books[books.language.notnull()].drop_duplicates(['book_author'])
look_up_df = look_up_df.set_index('book_author')['language']
books['language'] = books['language'].fillna(books['book_author'].map(look_up_df))
books=books.reset_index(drop=True)
print(books.isna().sum())


isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               25349
category               68851
summary                67227
img_path                   0
category_low           68851
dtype: int64


In [12]:
books[books.category.isna()].index

Int64Index([     5,      8,      9,     10,     11,     25,     27,     30,
                31,     42,
            ...
            149550, 149552, 149555, 149557, 149558, 149560, 149561, 149564,
            149567, 149569],
           dtype='int64', length=68851)

In [13]:
books.book_author

0         richardbrucewright
1                carlod'este
2             ginabarikolata
3                     amytan
4               robertcowley
                 ...        
149565          p.j.o'rourke
149566           samlightner
149567          claudedooley
149568           jeremylloyd
149569                mapsco
Name: book_author, Length: 149570, dtype: object

#### 책제목과 작가가 같으면 같은 책이므로 language가 있는 다른 index에서 language를 가져옴.

In [14]:
# category&summary 결측치 채우기
# 책제목과 작가가 같으면 같은 책이므로 language가 있는 다른 index에서 language를 가져옴.

def new_func(idx_n):
    return books.book_title.isin([books.iloc[idx_n].book_title]) & books.book_author.isin([books.iloc[idx_n].book_author])

for idx_n in books[books.category.isna()].index:
    # print(idx_n)
    books[new_func(idx_n)].ffill().bfill()
    books[new_func(idx_n)] = books[new_func(idx_n)].ffill().bfill()

5
8
9
10
11
25
27
30
31
42
51
52
55
60
61
62
66
72
76
77
78
79
80
85
90
93
94
100
102
104
106
107
110
111
113
116
118
120
125
126
130
133
138
141
145
147
151
152
158
162
165
168
171
172
173
174
176
177
178
179
180
181
182
183
184
185
186
187
189
190
191
192
193
194
199
200
201
202
203
204
205
206
207
211
215
218
219
221
223
227
228
231
232
237
238
240
241
242
243
252
254
255
256
257
260
264
268
271
273
279
280
281
282
287
293
299
300
302
310
313
316
317
321
322
323
325
330
331
333
338
343
345
346
347
349
350
351
352
355
356
357
358
360
367
368
369
375
376


KeyboardInterrupt: 

In [15]:
books.to_csv('new_books.csv', index=False)

In [16]:
# 확인
books[books.book_title.isin([books.iloc[25].book_title]) & books.book_author.isin([books.iloc[25].book_author])]

,isbn,book_title,book_author,year_of_publication,publisher,img_url,language,category,summary,img_path,category_low
25,0312978383,Winter Solstice,rosamundepilcher,2001.0,St. Martin's Paperbacks,http://images.amazon.com/images/P/0312978383.0...,en,fiction,Five people buffeted by life's difficulties co...,images/0312978383.01.THUMBZZZ.jpg,fiction
7264,0312244266,Winter Solstice,rosamundepilcher,2000.0,Thomas Dunne Books,http://images.amazon.com/images/P/0312244266.0...,en,fiction,Five people buffeted by life's difficulties co...,images/0312244266.01.THUMBZZZ.jpg,fiction
79374,B00009NDAN,Winter Solstice,rosamundepilcher,2000.0,St. Martin's Press,http://images.amazon.com/images/P/B00009NDAN.0...,en,fiction,Five people buffeted by life's difficulties co...,images/B00009NDAN.01.THUMBZZZ.jpg,fiction


### Category 결측값 fiction으로 채우기

In [17]:
# books['category'] 결측치
## fiction이 카테고리 50프로 이상이라 결측치 일단 fiction으로
books['category'] = books['category'].fillna("['Fiction']")

In [18]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               25349
category                   0
summary                67163
img_path                   0
category_low           68786
dtype: int64

In [30]:
for i in range(len(isbns)):
    books.loc[books["isbn"].str.startswith(str(isbns.iloc[i][0])), "country"] = isbns.iloc[i][1] # isbn으로 나라 못찾음!!

In [35]:
books = books[~(books.country.isna())] #여기서 드랍했어요!!!!!!!! - 

In [33]:
books.country.dropna(inplace = True)

In [46]:
books['language'].value_counts() * 100 / len(books[~books['language'].isna()])
# ca: 캐나다식 영어
# 

en       95.725146
de        1.556921
es        1.235093
fr        1.072357
it        0.149377
nl        0.081368
pt        0.057079
da        0.044935
ca        0.027932
ms        0.012144
no        0.007287
zh-CN     0.003643
ja        0.003643
gl        0.003643
ru        0.003643
la        0.003643
el        0.001214
th        0.001214
ro        0.001214
fa        0.001214
eo        0.001214
gd        0.001214
ga        0.001214
vi        0.001214
zh-TW     0.001214
ko        0.001214
Name: language, dtype: float64

In [41]:
len(books['language'].value_counts())

26

In [38]:
books.isna().sum()

isbn                       0
book_title                 0
book_author                0
year_of_publication        0
publisher                  0
img_url                    0
language               67179
category               68803
summary                67179
img_path                   0
country                    0
dtype: int64

In [62]:
pd.DataFrame(books['country'].value_counts()*100/len(books)).head(60)

,country
English,89.890383
German,4.484989
French,2.277272
Spain,2.273259
Italian,0.322363
Argentina,0.161850
Dutch/Flemish,0.117709
Portugal,0.070893
Mexico,0.063536
Japan,0.053504


# 유저

In [64]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 68092 entries, 0 to 68091
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   user_id   68092 non-null  int64  
 1   location  68092 non-null  object 
 2   age       40259 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 1.6+ MB


In [ ]:
language_dict = { 'English': 1 }